In [1]:
import cv2
import mediapipe as mp
import copy
import T_Functions as fn
import T_HandPoint as hp
import time
import pyshine as ps

Warning! No module named 'sounddevice'


In [2]:
min_detection_confidence = 0.75
min_tracking_confidence = 0.5
mpHands = mp.solutions.hands
hands = mpHands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=min_detection_confidence,
            min_tracking_confidence=min_tracking_confidence
        )
mpDraw = mp.solutions.drawing_utils

In [3]:
LbPath = './model/Hand_Point/HandPose.csv'
pose = fn.ReadLb(LbPath)
hpc = hp.HPC()

In [4]:
import numpy as np

In [5]:
cap = cv2.VideoCapture(0)
i = 0
j = 0
ip = -1
text =  'Input: '
while True:
    success,img=cap.read()
    img = cv2.flip(img, 1)

    # img = cv2.medianBlur(img,5)
    
    List = []
    textL, textR  = '', ''

    debug_img = copy.deepcopy(img)
    imgRGB = cv2.cvtColor(debug_img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks
        # for id, lm in enumerate(myHand):
        #     mpDraw.draw_landmarks(debug_img, lm, mpHands.HAND_CONNECTIONS, 
        #                             mpDraw.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
        #                             mpDraw.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
        #                             )
        
        for lm, handedness in zip(myHand, results.multi_handedness):
            brect = fn.CalcBR(debug_img, lm)
            lml = fn.FindPoint(debug_img, lm)
            pre = fn.pre_process(lml, handedness.classification[0].index)
            hand_sign_id, test = hpc(pre)
            debug_img = fn.DrawBR(True, debug_img, brect)

            if test[hand_sign_id] >= 0.5:
                debug_img = fn.DrawIT(
                            debug_img,
                            brect,
                            handedness,
                            pose[hand_sign_id]
                        )
                if handedness.classification[0].index == 1:
                    textR = 'R: ' + pose[hand_sign_id]
                else:
                    textL = 'L: ' + pose[hand_sign_id]
                # if ip != hand_sign_id:
                #     ip = hand_sign_id
                #     j = 0
                # else:
                #     if j == 15:
                #         text  =  text + pose[hand_sign_id]
                #         j = 0
                #     else: 
                #         j += 1
            else:
                debug_img = fn.DrawIT(
                            debug_img,
                            brect,
                            handedness,
                            ""
                        )
        # #if test[hand_sign_id] >= 0.99999:
        # print(test[hand_sign_id])
    text = textL + '     ' + textR
    cv2.rectangle(debug_img, (10, 10), (300, 55), (0, 207, 0), -1)
    cv2.putText(debug_img, text, (20, 45),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1.3, (245, 0, 0), 3, cv2.LINE_AA)
    # debug_img = ps.putBText(debug_img, text, text_offset_x=20, text_offset_y=20, vspace=10, hspace=10, font_scale=1.0, background_RGB=(228,225,222), text_RGB=(1,1,1))
    # cv2.rectangle(debug_img, (10, 10), (600, 100), (121, 22, 76), -1)
    # cv2.putText(debug_img, "info_text", (52, 52),
    #            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.imshow("Image",debug_img)

    key = cv2.waitKey(10)
    if key == 27:
        break
    elif key >= 97 & key <= 122:
        i+=1
        if i > 5:
            if results.multi_hand_landmarks:
                myHand = results.multi_hand_landmarks
                #List1 = fn.FindPoint(results.multi_hand_landmarks)
                #List = fn.pre_process(List1)
                fn.SavePoint(key, pre)

                        
cap.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
cv2.destroyAllWindows()